<a href="https://colab.research.google.com/github/kessingtonosazee/GCP_Project_1/blob/master/Code_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [211]:
# Install packages
!pip install pulp


In [245]:
# Import Libraries
import numpy as np
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable


In [246]:
# Load data
wards = ["ward1", "ward2", "ward3"]
max_beds_per_ward = {"ward1": 60, "ward2": 50, "ward3": 50}

In [247]:
# Simulate uncertain demand using a normal distribution
np.random.seed(42)  # Set seed for reproducibility
demand_mean = {"ward1": 10, "ward2": 10, "ward3": 10}
demand_std_dev = {"ward1": 3, "ward2": 2, "ward3": 4}
demand = {ward: max(0, int(np.random.normal(demand_mean[ward], demand_std_dev[ward]))) for ward in wards}

In [215]:
# Priority for each ward (you can adjust these values)
# ward_priority = {"ward1": 2, "ward2": 1, "ward3": 3}

In [248]:
# Real-time bed availability
real_time_availability = {"ward1": 40, "ward2":30, "ward3": 30}  # Replace with actual real-time data


In [249]:
# Priority as a function of demand and availability
# def ward_priority(ward):
#     # Define your priority function here
#     # This is a simple example; you should tailor it to your specific needs
#     # In this example, priority is higher when demand is higher and availability is lower
#     return demand[ward]  # Adding 1 to avoid division by zero

def ward_priority(ward):
    # Define custom weights based on specific criteria
    weights = {"ward1": 1.0, "ward2": 0.5, "ward3": 0.2}
    return weights[ward]


In [252]:
# Create a linear programming problem
model = LpProblem(name="Healthcare_Bed_Mininmum_Allocation", sense=LpMinimize)

In [253]:
# Decision Variables: Number of beds assigned to each ward
beds = {ward: LpVariable(name=f"{ward}_beds", lowBound=0, cat="Integer") for ward in wards}

In [254]:
# Objective Function: Maximize the total weighted number of beds
model += sum(ward_priority(ward) * beds[ward] for ward in wards), "Total_Weighted_Beds"

In [255]:
# Constraints: Demand must be met for each ward, and the number of beds cannot exceed the maximum for each ward
for ward in wards:
    model += beds[ward] >= demand[ward], f"Ward_Demand_{ward}"
    model += beds[ward] <= max_beds_per_ward[ward], f"Max_Bed_{ward}"
    model += beds[ward] <= real_time_availability[ward], f"Real_Time_Availability_{ward}"


In [256]:
# Solve the linear programming problem
model.solve()

1

In [257]:
# Display the results
if model.status == 1:  # Optimal solution found
    for ward in wards:
        print(f"Minimum number of beds for {ward}: {beds[ward].varValue}")
else:
    print("No optimal solution found.")


Minimum number of beds for ward1: 11.0
Minimum number of beds for ward2: 9.0
Minimum number of beds for ward3: 12.0
